<a href="https://colab.research.google.com/github/candenizkocak/THYAO-Price-Prediction/blob/main/THYAO_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
import yfinance as yf

import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tensorflow import keras

from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

## Loading Data

In [ ]:
stock_symbol = 'THYAO.IS'

start_date = datetime.datetime(2021, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

In [ ]:
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
stock_data = stock_data.reset_index()

In [ ]:
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,12.970000,12.970000,12.670000,12.680000,12.680000,59948726
1,2021-01-05,12.640000,12.800000,12.520000,12.580000,12.580000,90905126
2,2021-01-06,12.730000,12.810000,12.630000,12.680000,12.680000,63226286
3,2021-01-07,12.740000,12.760000,12.620000,12.680000,12.680000,63579108
4,2021-01-08,12.750000,12.960000,12.730000,12.840000,12.840000,115414243
...,...,...,...,...,...,...,...
496,2022-12-26,143.300003,145.300003,142.800003,145.000000,145.000000,43689692
497,2022-12-27,145.199997,146.600006,143.000000,143.100006,143.100006,63253933
498,2022-12-28,143.500000,143.800003,137.100006,140.100006,140.100006,48565337
499,2022-12-29,140.199997,143.199997,138.500000,142.300003,142.300003,45564894


In [ ]:
THYAO_training_complete = stock_data

THYAO_training_complete['Date'] = pd.to_datetime(THYAO_training_complete['Date'])

THYAO_training_complete.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,12.97,12.97,12.67,12.68,12.68,59948726
1,2021-01-05,12.64,12.80,12.52,12.58,12.58,90905126
2,2021-01-06,12.73,12.81,12.63,12.68,12.68,63226286
3,2021-01-07,12.74,12.76,12.62,12.68,12.68,63579108
4,2021-01-08,12.75,12.96,12.73,12.84,12.84,115414243
5,2021-01-11,12.79,12.90,12.65,12.70,12.70,68875132
6,2021-01-12,12.77,12.99,12.75,12.85,12.85,93572479
7,2021-01-13,12.97,13.28,12.97,13.26,13.26,212821900
8,2021-01-14,13.45,13.67,13.12,13.24,13.24,204872119
9,2021-01-15,13.14,13.39,13.06,13.28,13.28,126680673


## Data preprocessing

In [ ]:
THYAO_training_processed = THYAO_training_complete.iloc[:, 4:5].values
THYAO_training_processed

array([[ 12.68000031],
       [ 12.57999992],
       [ 12.68000031],
       [ 12.68000031],
       [ 12.84000015],
       [ 12.69999981],
       [ 12.85000038],
       [ 13.26000023],
       [ 13.23999977],
       [ 13.27999973],
       [ 13.23999977],
       [ 13.48999977],
       [ 13.26000023],
       [ 13.02999973],
       [ 12.93000031],
       [ 12.69999981],
       [ 12.63000011],
       [ 11.94999981],
       [ 12.32999992],
       [ 12.10999966],
       [ 12.01000023],
       [ 12.06000042],
       [ 12.14999962],
       [ 12.19999981],
       [ 12.15999985],
       [ 12.14999962],
       [ 12.17000008],
       [ 12.21000004],
       [ 12.14999962],
       [ 12.        ],
       [ 12.39999962],
       [ 12.18000031],
       [ 12.19999981],
       [ 12.27000046],
       [ 12.30000019],
       [ 12.64000034],
       [ 13.02000046],
       [ 12.82999992],
       [ 13.39000034],
       [ 13.48999977],
       [ 13.80000019],
       [ 14.81000042],
       [ 14.92000008],
       [ 14

In [ ]:
scaler = MinMaxScaler(feature_range = (0, 1))

THYAO_training_scaled = scaler.fit_transform(THYAO_training_processed)

In [ ]:
print(THYAO_training_scaled)
print(THYAO_training_scaled.shape)

[[0.01318519]
 [0.01244445]
 [0.01318519]
 [0.01318519]
 [0.01437037]
 [0.01333334]
 [0.01444445]
 [0.01748149]
 [0.01733334]
 [0.01762963]
 [0.01733334]
 [0.01918519]
 [0.01748149]
 [0.01577778]
 [0.01503704]
 [0.01333334]
 [0.01281482]
 [0.00777778]
 [0.0105926 ]
 [0.00896296]
 [0.00822223]
 [0.0085926 ]
 [0.00925926]
 [0.00962963]
 [0.00933334]
 [0.00925926]
 [0.00940741]
 [0.00970371]
 [0.00925926]
 [0.00814815]
 [0.01111111]
 [0.00948149]
 [0.00962963]
 [0.01014815]
 [0.01037038]
 [0.01288889]
 [0.01570371]
 [0.0142963 ]
 [0.01844445]
 [0.01918519]
 [0.02148149]
 [0.02896297]
 [0.02977778]
 [0.0282963 ]
 [0.02874074]
 [0.02777778]
 [0.03051853]
 [0.02962963]
 [0.02851852]
 [0.02666667]
 [0.02807407]
 [0.02844445]
 [0.02703704]
 [0.02251852]
 [0.02140741]
 [0.01125926]
 [0.00970371]
 [0.01562963]
 [0.01311112]
 [0.01303704]
 [0.0117037 ]
 [0.01340741]
 [0.01370371]
 [0.01555556]
 [0.01488889]
 [0.01600001]
 [0.01377778]
 [0.01444445]
 [0.01207408]
 [0.0077037 ]
 [0.00792593]
 [0.00

In [ ]:
features_set = []
labels = []
for i in range(60, THYAO_training_scaled.shape[0]):
    features_set.append(THYAO_training_scaled[i-60:i, 0])
    labels.append(THYAO_training_scaled[i, 0])

In [ ]:
features_set, labels = np.array(features_set), np.array(labels)

In [ ]:
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))
print(features_set.shape)

(441, 60, 1)


**LSTM layers work on 3D data with the following structure (nb_sequence, nb_timestep, nb_feature).

    nb_sequence corresponds to the total number of sequences in your dataset (or to the batch size if you are using mini-batch learning).
    nb_timestep corresponds to the size of your sequences.
    nb_feature corresponds to number of features describing each of your timesteps.


## RNN

#### Build RNN Model

In [ ]:
RNN_model = Sequential()
RNN_model.add(SimpleRNN(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100, return_sequences=True))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100, return_sequences=True))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100))
RNN_model.add(Dropout(0.2))

RNN_model.add(Dense(units = 1))

Compile RNN Model

In [ ]:
RNN_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
RNN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 60, 300)           90600     
                                                                 
 dropout (Dropout)           (None, 60, 300)           0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 60, 100)           40100     
                                                                 
 dropout_1 (Dropout)         (None, 60, 100)           0         
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 60, 100)           20100     
                                                                 
 dropout_2 (Dropout)         (None, 60, 100)           0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 100)               2

#### Fit RNN Model


In [ ]:
RNN_History = RNN_model.fit(features_set, labels, epochs = 300, batch_size = 32)

Epoch 1/300
14/14 [==============================] - 16s 417ms/step - loss: 0.8502
Epoch 2/300
14/14 [==============================] - 4s 286ms/step - loss: 0.5452
Epoch 3/300
14/14 [==============================] - 3s 218ms/step - loss: 0.5091
Epoch 4/300
14/14 [==============================] - 3s 181ms/step - loss: 0.4588
Epoch 5/300
14/14 [==============================] - 3s 183ms/step - loss: 0.3749
Epoch 6/300
14/14 [==============================] - 5s 337ms/step - loss: 0.4177
Epoch 7/300
14/14 [==============================] - 4s 306ms/step - loss: 0.3539
Epoch 8/300
14/14 [==============================] - 3s 206ms/step - loss: 0.3291
Epoch 9/300
14/14 [==============================] - 4s 289ms/step - loss: 0.3015
Epoch 10/300
14/14 [==============================] - 7s 425ms/step - loss: 0.3165
Epoch 11/300
14/14 [==============================] - 3s 179ms/step - loss: 0.2882
Epoch 12/300
14/14 [==============================] - 3s 190ms/step - loss: 0.2744
Epoch 13/300

## LSTM

####  Build LSTM Model

In [ ]:
LSTM_model = Sequential()
LSTM_model.add(LSTM(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100, return_sequences=True))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100, return_sequences=True))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(Dense(units = 1))

#### Compile LSTM Model

In [ ]:
LSTM_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
LSTM_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 300)           362400    
                                                                 
 dropout_4 (Dropout)         (None, 60, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 100)           160400    
                                                                 
 dropout_5 (Dropout)         (None, 60, 100)           0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 100)           80400     
                                                                 
 dropout_6 (Dropout)         (None, 60, 100)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)              

#### Fit LSTM Model

In [ ]:
LSTM_History = LSTM_model.fit(features_set, labels, epochs = 300, batch_size = 32)

Epoch 1/300
14/14 [==============================] - 8s 25ms/step - loss: 0.0420
Epoch 2/300
14/14 [==============================] - 0s 15ms/step - loss: 0.0071
Epoch 3/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0030
Epoch 4/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0023
Epoch 5/300
14/14 [==============================] - 0s 15ms/step - loss: 0.0019
Epoch 6/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0020
Epoch 7/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0013
Epoch 8/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0018
Epoch 9/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0020
Epoch 10/300
14/14 [==============================] - 0s 16ms/step - loss: 0.0023
Epoch 11/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0023
Epoch 12/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0016
Epoch 13/300
14/14 [=====

## GRU

####  Build GRU Model

In [ ]:
GRU_model = Sequential()
GRU_model.add(GRU(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100, return_sequences=True))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100, return_sequences=True))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100))
GRU_model.add(Dropout(0.2))

GRU_model.add(Dense(units = 1))

#### Compile GRU Model

In [ ]:
GRU_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
GRU_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 60, 300)           272700    
                                                                 
 dropout_8 (Dropout)         (None, 60, 300)           0         
                                                                 
 gru_1 (GRU)                 (None, 60, 100)           120600    
                                                                 
 dropout_9 (Dropout)         (None, 60, 100)           0         
                                                                 
 gru_2 (GRU)                 (None, 60, 100)           60600     
                                                                 
 dropout_10 (Dropout)        (None, 60, 100)           0         
                                                                 
 gru_3 (GRU)                 (None, 100)              

#### Fit GRU Model

In [ ]:
GRU_History = GRU_model.fit(features_set, labels, epochs = 300, batch_size = 32)

Epoch 1/300
14/14 [==============================] - 7s 26ms/step - loss: 0.0254
Epoch 2/300
14/14 [==============================] - 0s 19ms/step - loss: 0.0038
Epoch 3/300
14/14 [==============================] - 0s 19ms/step - loss: 0.0031
Epoch 4/300
14/14 [==============================] - 0s 20ms/step - loss: 0.0018
Epoch 5/300
14/14 [==============================] - 0s 18ms/step - loss: 0.0016
Epoch 6/300
14/14 [==============================] - 0s 19ms/step - loss: 0.0012
Epoch 7/300
14/14 [==============================] - 0s 19ms/step - loss: 0.0015
Epoch 8/300
14/14 [==============================] - 0s 17ms/step - loss: 0.0015
Epoch 9/300
14/14 [==============================] - 0s 18ms/step - loss: 0.0018
Epoch 10/300
14/14 [==============================] - 0s 18ms/step - loss: 0.0015
Epoch 11/300
14/14 [==============================] - 0s 18ms/step - loss: 0.0018
Epoch 12/300
14/14 [==============================] - 0s 14ms/step - loss: 0.0014
Epoch 13/300
14/14 [=====

## Evaluate The Model

#### Evalute Model Training

In [ ]:
fig = go.Figure()

# Training loss trace
fig.add_trace(go.Scatter(x=list(range(1, len(RNN_History.history['loss']) + 1)),
                         y=RNN_History.history['loss'],
                         mode='lines',
                         name='RNN Training Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(LSTM_History.history['loss']) + 1)),
                         y=LSTM_History.history['loss'],
                         mode='lines',
                         name='LSTM Training Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(GRU_History.history['loss']) + 1)),
                         y=GRU_History.history['loss'],
                         mode='lines',
                         name='GRU Training Loss'))

fig.update_layout(title='Training and Validation Loss',
                  xaxis_title='Epoch',
                  yaxis_title='Loss',
                  legend_title='Loss Type',
                  hovermode='x',
                  hoverlabel=dict(bgcolor='white', font_size=12, font_family='Rockwell'),
                  template='plotly_white')

fig.show()

#### Evalute Model Prediction

In [ ]:
test_stock_symbol = 'THYAO.IS'

start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime.now() - datetime.timedelta(days=1)
test_stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
test_stock_data = stock_data.reset_index()

THYAO_testing_complete = test_stock_data
THYAO_testing_processed = THYAO_testing_complete.iloc[:, 1:2].values

THYAO_total = pd.concat((THYAO_training_complete['Close'], THYAO_testing_complete['Close']), axis=0)

test_inputs = THYAO_total[len(THYAO_total) - len(THYAO_testing_complete) - 60:].values
test_inputs

[*********************100%%**********************]  1 of 1 completed


array([ 86.15000153,  84.5       ,  85.09999847,  88.        ,
        90.        ,  98.44999695,  97.25      ,  97.40000153,
        99.05000305,  97.94999695,  99.5       , 100.5       ,
        98.80000305,  99.40000153,  96.94999695, 101.30000305,
       102.40000153, 100.80000305, 101.80000305, 104.19999695,
       107.19999695, 113.        , 111.59999847, 113.40000153,
       111.5       , 109.69999695, 109.        , 104.59999847,
       107.19999695, 106.59999847, 106.90000153, 111.19999695,
       112.5       , 111.59999847, 113.80000305, 115.09999847,
       118.90000153, 120.        , 124.40000153, 128.30000305,
       138.6000061 , 135.6000061 , 134.69999695, 135.3999939 ,
       141.        , 145.8999939 , 144.        , 137.        ,
       140.1000061 , 139.19999695, 143.30000305, 145.8999939 ,
       142.8999939 , 142.1000061 , 141.80000305, 145.        ,
       143.1000061 , 140.1000061 , 142.30000305, 140.8999939 ,
        12.68000031,  12.57999992,  12.68000031,  12.68

In [ ]:
test_inputs = test_inputs.reshape(-1,1)
test_inputs = scaler.transform(test_inputs)

test_features = []
for i in range(60, len(test_inputs)):
    test_features.append(test_inputs[i-60:i, 0])

test_features = np.array(test_features)
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))
test_features.shape

(501, 60, 1)

In [ ]:
# Convert datetime values to timestamps
THYAO_testing_processed_timestamps = pd.to_datetime(THYAO_testing_processed).view(int) // 10**9

# Now, you can use the converted timestamps for evaluation
RNN_test_loss = RNN_model.evaluate(test_features, THYAO_testing_processed_timestamps)
print(f'RNN Test Loss: {RNN_test_loss}')

LSTM_test_loss = LSTM_model.evaluate(test_features, THYAO_testing_processed_timestamps)
print(f'LSTM Test Loss: {LSTM_test_loss}')

GRU_test_loss = GRU_model.evaluate(test_features, THYAO_testing_processed_timestamps)
print(f'GRU Test Loss: {GRU_test_loss}')


16/16 [==============================] - 1s 32ms/step - loss: 2693704257126793216.0000
RNN Test Loss: 2.693704257126793e+18
16/16 [==============================] - 2s 12ms/step - loss: 2693704257126793216.0000
LSTM Test Loss: 2.693704257126793e+18
16/16 [==============================] - 1s 10ms/step - loss: 2693704257126793216.0000
GRU Test Loss: 2.693704257126793e+18


## Make predictions

In [ ]:
# RNN
RNN_predictions = RNN_model.predict(test_features)
RNN_predictions = scaler.inverse_transform(RNN_predictions)

# LSTM
LSTM_predictions = LSTM_model.predict(test_features)
LSTM_predictions = scaler.inverse_transform(LSTM_predictions)

# GRU
GRU_predictions = GRU_model.predict(test_features)
GRU_predictions = scaler.inverse_transform(GRU_predictions)

16/16 [==============================] - 1s 7ms/step


In [ ]:
THYAO_testing_complete

,index,Date,Open,High,Low,Close,Adj Close,Volume
0,0,2021-01-04,12.970000,12.970000,12.670000,12.680000,12.680000,59948726
1,1,2021-01-05,12.640000,12.800000,12.520000,12.580000,12.580000,90905126
2,2,2021-01-06,12.730000,12.810000,12.630000,12.680000,12.680000,63226286
3,3,2021-01-07,12.740000,12.760000,12.620000,12.680000,12.680000,63579108
4,4,2021-01-08,12.750000,12.960000,12.730000,12.840000,12.840000,115414243
...,...,...,...,...,...,...,...,...
496,496,2022-12-26,143.300003,145.300003,142.800003,145.000000,145.000000,43689692
497,497,2022-12-27,145.199997,146.600006,143.000000,143.100006,143.100006,63253933
498,498,2022-12-28,143.500000,143.800003,137.100006,140.100006,140.100006,48565337
499,499,2022-12-29,140.199997,143.199997,138.500000,142.300003,142.300003,45564894


In [ ]:
THYAO_prediction = THYAO_testing_complete['Date']
THYAO_prediction = pd.DataFrame(THYAO_prediction)
THYAO_prediction['RNN_Open'] = RNN_predictions
THYAO_prediction['LSTM_Open'] = LSTM_predictions
THYAO_prediction['GRU_Open'] = GRU_predictions

THYAO_prediction

,Date,RNN_Open,LSTM_Open,GRU_Open
0,2021-01-04,138.224487,148.379639,143.704941
1,2021-01-05,136.826355,120.118980,80.355087
2,2021-01-06,133.172699,94.298210,17.403555
3,2021-01-07,113.617401,72.115280,21.659712
4,2021-01-08,99.893791,60.559193,21.168291
...,...,...,...,...
496,2022-12-26,138.197235,148.650513,143.567337
497,2022-12-27,138.212906,149.546326,145.632339
498,2022-12-28,138.222733,149.451920,144.445374
499,2022-12-29,138.224991,148.387955,142.351501


In [ ]:
fig = go.Figure()

# Actual THYAO stock price trace
fig.add_trace(go.Scatter(x=THYAO_testing_complete.Date,
                         y=THYAO_testing_complete.Open,
                         mode='lines',
                         name='Actual THYAO Stock Price',
                         line=dict(color='blue')))

# RNN Predicted THYAO stock price trace
fig.add_trace(go.Scatter(x=THYAO_prediction.Date,
                         y=THYAO_prediction.RNN_Open,
                         mode='lines',
                         name='RNN Predicted THYAO Stock Price',
                         line=dict(color='red')))

# LSTM Predicted THYAO stock price trace
fig.add_trace(go.Scatter(x=THYAO_prediction.Date,
                         y=THYAO_prediction.LSTM_Open,
                         mode='lines',
                         name='LSTM Predicted THYAO Stock Price',
                         line=dict(color='green')))

# LSTM Predicted THYAO stock price trace
fig.add_trace(go.Scatter(x=THYAO_prediction.Date,
                         y=THYAO_prediction.GRU_Open,
                         mode='lines',
                         name='GRU Predicted THYAO Stock Price',
                         line=dict(color='yellow')))

fig.update_layout(title='THYAO Stock Price Prediction',
                  xaxis_title='Date',
                  yaxis_title='THYAO Stock Price',
                  legend_title='Price Type',
                  hovermode='x',
                  hoverlabel=dict(bgcolor='white', font_size=12, font_family='Rockwell'),
                  template='plotly_white')

fig.show()